In [1]:
!uv --version

uv 0.9.5 (d5f39331a 2025-10-21)


In [2]:
!uv init

Initialized project `jw`


In [3]:
!uv add scikit-learn==1.6.1

Using CPython 3.13.5 interpreter at: /opt/anaconda3/bin/python3
Creating virtual environment at: .venv
Resolved 6 packages in 206ms                                         
Prepared 5 packages in 816ms                                             
Installed 5 packages in 53ms                                
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.2
 + threadpoolctl==3.6.0


In [4]:
!cat uv.lock

version = 1
revision = 3
requires-python = ">=3.13"

[[package]]
name = "joblib"
version = "1.5.2"
source = { registry = "https://pypi.org/simple" }
sdist = { url = "https://files.pythonhosted.org/packages/e8/5d/447af5ea094b9e4c4054f82e223ada074c552335b9b4b2d14bd9b35a67c4/joblib-1.5.2.tar.gz", hash = "sha256:3faa5c39054b2f03ca547da9b2f52fde67c06240c31853f306aea97f13647b55", size = 331077, upload-time = "2025-08-27T12:15:46.575Z" }
wheels = [
    { url = "https://files.pythonhosted.org/packages/1e/e8/685f47e0d754320684db4425a0967f7d3fa70126bffd76110b7009a0090f/joblib-1.5.2-py3-none-any.whl", hash = "sha256:4e1f0bdbb987e6d843c70cf43714cb276623def372df3c22fe5266b2670bc241", size = 308396, upload-time = "2025-08-27T12:15:45.188Z" },
]

[[package]]
name = "jw"
version = "0.1.0"
source = { virtual = "." }
dependencies = [
    { name = "scikit-learn" },
]

[package.metadata]
requires-dist = [{ name = "scikit-learn", specifier = "==1.6.1" }]

[[package]]
name = "numpy"
version = "2.3.4"
source

In [6]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

/bin/bash: wget: command not found


In [7]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

/bin/bash: wget: command not found


In [9]:
!curl -L -o pipeline_v1.bin https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1300  100  1300    0     0   3211      0 --:--:-- --:--:-- --:--:--  3211


In [14]:
import pickle

with open('pipeline_v1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

print(type(model))

<class 'sklearn.pipeline.Pipeline'>


In [16]:
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
y_pred = model.predict_proba([record])
print("Predicted probability:", y_pred)

Predicted probability: [[0.46639273 0.53360727]]


In [17]:
print( model.classes_)
print(model.predict_proba([record]))

[0 1]
[[0.46639273 0.53360727]]


In [1]:
!pip install fastapi "uvicorn[standard]" -q

In [3]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.testclient import TestClient

# Define the input data model
class Lead(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# Load the trained model
with open("pipeline_v1.bin", "rb") as f:
    model = pickle.load(f)
print("✅ Model loaded:", type(model))

# Create the FastAPI app
app = FastAPI()

# Define prediction endpoint
@app.post("/predict")
def predict(lead: Lead):
    X = [lead.model_dump()]
    proba = float(model.predict_proba(X)[0, 1])
    return {"probability": proba}

# Test the endpoint locally using TestClient
client = TestClient(app)
payload = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
r = client.post("/predict", json=payload)
print("HTTP status code:", r.status_code)
print("Response JSON:", r.json())

✅ Model loaded: <class 'sklearn.pipeline.Pipeline'>
HTTP status code: 200
Response JSON: {'probability': 0.5340417283801275}


In [4]:
!docker --version

Docker version 28.5.1, build e180ab8


In [5]:
!docker pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model

9c0578b5: Pulling fs layer 
357cf7a1: Pulling fs layer 
f5177ae2: Pulling fs layer 
2b9cc6ee: Pulling fs layer 
2667830b: Pulling fs layer 
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [6]:
!docker images

REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      14d79fde0bbf   5 days ago   181MB


In [11]:
!docker build -t zoomcamp-model-app .


[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (8/9)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 233B                                       0.0s
 => WARN: InvalidBaseImagePlatform: Base image agrigorev/zoomcamp-model:2  0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:2025   0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/5] FROM docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bb  0.0s
 => => resolve docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bb  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 1.37kB                                        0.0s
                                       

In [15]:
!docker run -p 8000:8000 zoomcamp-model-app

Traceback (most recent call last):
  File "/usr/local/bin/uvicorn", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1462, in __call__
    return self.main(*args, **kwargs)
           ~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1383, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 1246, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.13/site-packages/click/core.py", line 814, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.13/site-packages/uvicorn/main.py", line 423, in main
    run(
    ~~~^
        app,
        ^^^^
    ...<46 lines>...
        h11_max_incomplete_event_size=h11_max_incomplete_event_size,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    

In [16]:
!docker build --platform=linux/amd64 --no-cache -t zoomcamp-model-app:v2 

ERROR: docker: 'docker buildx build' requires 1 argument

Usage:  docker buildx build [OPTIONS] PATH | URL | -

Run 'docker buildx build --help' for more information


In [17]:
!docker build --platform=linux/amd64 --no-cache -t zoomcamp-model-app:v2 .


[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (8/9)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 269B                                       0.0s
 => [internal] load metadata for docker.io/agrigorev/zoomcamp-model:2025   0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/5] FROM docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bb  0.0s
 => => resolve docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bb  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 63B                                           0.0s
 => CACHED [2/5] WORKDIR /app                                              0.0s
 => [3/5] COPY app.py .                

In [18]:
!docker run --rm --platform=linux/amd64 zoomcamp-model-app:v2 python -c "import sklearn; print('OK, sklearn version:', sklearn.__version__)"

OK, sklearn version: 1.6.1


In [19]:
!docker run --platform=linux/amd64 -p 8010:8000 zoomcamp-model-app:v2

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
^C
INFO:     Shutting down


In [20]:
import requests

url = "http://127.0.0.1:8010/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
print(requests.post(url, json=client).json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8010): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15efa0440>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
!docker ps

In [21]:
!docker ps -a
!docker rm -f $(docker ps -aq) 

CONTAINER ID   IMAGE                   COMMAND                  CREATED          STATUS                      PORTS     NAMES
3cfb65cf3acb   zoomcamp-model-app:v2   "uvicorn app:app --h…"   15 minutes ago   Exited (0) 5 minutes ago              elegant_ride
a139c0eed65d   zoomcamp-model-app      "uvicorn app:app --h…"   26 minutes ago   Exited (1) 23 minutes ago             xenodochial_leakey
024ceb7c0ed2   zoomcamp-model-app      "uvicorn app:app --h…"   26 minutes ago   Exited (1) 24 minutes ago             gifted_hawking
4f6b3584551f   zoomcamp-model-app      "uvicorn app:app --h…"   26 minutes ago   Exited (1) 24 minutes ago             musing_ritchie
5d815fb4ed64   zoomcamp-model-app      "uvicorn app:app --h…"   44 minutes ago   Exited (1) 42 minutes ago             beautiful_fermi
94e8a6426746   66432c08b81b            "uvicorn app:app --h…"   20 hours ago     Exited (1) 20 hours ago               vibrant_bhaskara
3cfb65cf3acb
a139c0eed65d
024ceb7c0ed2
4f6b3584551f
5d815fb4ed64
9

In [22]:
!docker run -d --platform=linux/amd64 -p 8010:8000 --name zoomcamp-app zoomcamp-model-app:v2

8db7617338fc3a3db0195a936e9bc562cac4f4cc277e709b456c2d19147bb975


In [23]:
!docker logs zoomcamp-app --tail 30

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [24]:
!docker ps --filter name=zoomcamp-app --format "table {{.Names}}\t{{.Ports}}\t{{.Status}}"

{.Names}   {.Ports}   {.Status}
{.Names}   {.Ports}   {.Status}


In [25]:
!docker run -d --platform=linux/amd64 -p 8010:8000 --name zoomcamp-app zoomcamp-model-app:v2

docker: Error response from daemon: Conflict. The container name "/zoomcamp-app" is already in use by container "8db7617338fc3a3db0195a936e9bc562cac4f4cc277e709b456c2d19147bb975". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


In [26]:
!docker ps

CONTAINER ID   IMAGE                   COMMAND                  CREATED         STATUS         PORTS                                         NAMES
8db7617338fc   zoomcamp-model-app:v2   "uvicorn app:app --h…"   3 minutes ago   Up 3 minutes   0.0.0.0:8010->8000/tcp, [::]:8010->8000/tcp   zoomcamp-app


In [27]:
import requests

url = "http://127.0.0.1:8010/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

r = requests.post(url, json=client, timeout=5)
print(r.status_code, r.json())

200 {'probability': 0.5340417283801275}


In [28]:
!docker stop zoomcamp-app
!docker rm zoomcamp-app

zoomcamp-app
zoomcamp-app
